# Model Comparison
## Find the Best Model



In [2]:
from pprint import pprint
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from glob import glob
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
!pwd

/Users/chris/github/reddit_nlp


In [ ]:
# === TODO === #
# use postgres to store and retrieve the compare df rows

In [5]:
glob('data/compare_df/*.csv')

[]

In [9]:
df = pd.read_csv('../data/compare_df/2020-05-01_1200.csv')

In [13]:
df.drop(columns='Unnamed: 0', inplace=True)
df.head(10)

,date,preprocessor,estimator,best_params,best_train_score,best_test_score,variance,prep_code,est_code,subreddits,fit_and_score_time
0,2020-05-01 12:00:35.796157,CountVectorizer,Logistic Regression,"{'count_vec__max_features': 5000, 'count_vec__...",0.981552,0.969261,1.252153,count_vec,logreg,"css, html, machinelearning, python",33.522334
1,2020-05-01 12:01:19.631395,TfidVectorizer,Logistic Regression,"{'logreg__C': 3, 'logreg__max_iter': 1000, 'lo...",0.975797,0.962296,1.383680,tfidf,logreg,"css, html, machinelearning, python",43.830402
2,2020-05-01 12:03:09.137769,CountVectorizer,Random Forest,"{'count_vec__max_features': 5000, 'count_vec__...",0.932162,0.920200,1.283301,count_vec,randomforest,"css, html, machinelearning, python",109.501659
3,2020-05-01 12:06:42.722919,TfidVectorizer,Random Forest,"{'randomforest__max_depth': 200, 'randomforest...",0.933904,0.923910,1.070126,tfidf,randomforest,"css, html, machinelearning, python",213.579916
4,2020-05-01 12:09:18.514413,CountVectorizer,K Nearest Neighbors,"{'count_vec__max_features': 5000, 'count_vec__...",0.980971,0.957147,2.428608,count_vec,knearest,"css, html, machinelearning, python",155.786147
5,2020-05-01 12:12:37.650303,TfidVectorizer,K Nearest Neighbors,"{'knearest__metric': 'manhattan', 'knearest__n...",0.977842,0.947910,3.060961,tfidf,knearest,"css, html, machinelearning, python",199.131211
6,2020-05-01 12:12:41.978814,CountVectorizer,Multinomial Bayes Classifier,"{'count_vec__max_features': 5000, 'count_vec__...",0.898723,0.881208,1.948836,count_vec,multinomialnb,"css, html, machinelearning, python",4.323703
7,2020-05-01 12:12:48.859678,TfidVectorizer,Multinomial Bayes Classifier,"{'multinomialnb__alpha': 0.01, 'multinomialnb_...",0.922875,0.903089,2.143951,tfidf,multinomialnb,"css, html, machinelearning, python",6.876112
8,2020-05-01 12:15:18.392077,CountVectorizer,Support Vector Classifier,"{'count_vec__max_features': 5000, 'count_vec__...",0.987659,0.978044,0.973553,count_vec,svc,"css, html, machinelearning, python",149.527599
9,2020-05-01 12:18:16.029289,TfidVectorizer,Support Vector Classifier,"{'svc__C': 4, 'svc__degree': 1, 'svc__gamma': ...",0.988896,0.980769,0.821764,tfidf,svc,"css, html, machinelearning, python",177.632564


In [19]:
columns_of_interest = ['preprocessor','estimator','best_test_score','fit_and_score_time','time_weighted_score']

In [21]:
df.sort_values(by='time_weighted_score', ascending=False)[columns_of_interest]

,preprocessor,estimator,best_test_score,fit_and_score_time,time_weighted_score
20,CountVectorizer,Passive Agressive Classifier,0.972895,2.821999,344.753881
21,TfidVectorizer,Passive Agressive Classifier,0.973879,4.145939,234.899593
6,CountVectorizer,Multinomial Bayes Classifier,0.881208,4.323703,203.808737
22,CountVectorizer,Stochastic Gradient Descent Classifier,0.951923,5.427454,175.390369
7,TfidVectorizer,Multinomial Bayes Classifier,0.903089,6.876112,131.337153
23,TfidVectorizer,Stochastic Gradient Descent Classifier,0.927771,8.338774,111.259890
10,CountVectorizer,AdaBoost Classifier,0.450182,9.999811,45.019023
0,CountVectorizer,Logistic Regression,0.969261,33.522334,28.913890
11,TfidVectorizer,AdaBoost Classifier,0.450030,18.964186,23.730536
1,TfidVectorizer,Logistic Regression,0.962296,43.830402,21.954979


In [26]:
df.sort_values(by='best_test_score', ascending=False)[columns_of_interest]

,preprocessor,estimator,best_test_score,fit_and_score_time,time_weighted_score
15,TfidVectorizer,Extra Trees Classifier,0.981072,510.018418,1.923601
9,TfidVectorizer,Support Vector Classifier,0.980769,177.632564,5.521337
14,CountVectorizer,Extra Trees Classifier,0.979558,361.378415,2.710615
12,CountVectorizer,Bagging Classifier,0.979104,148.539304,6.591545
13,TfidVectorizer,Bagging Classifier,0.978271,315.586956,3.099845
8,CountVectorizer,Support Vector Classifier,0.978044,149.527599,6.540890
21,TfidVectorizer,Passive Agressive Classifier,0.973879,4.145939,234.899593
20,CountVectorizer,Passive Agressive Classifier,0.972895,2.821999,344.753881
0,CountVectorizer,Logistic Regression,0.969261,33.522334,28.913890
1,TfidVectorizer,Logistic Regression,0.962296,43.830402,21.954979
